# Distance: Flying Experiments

Detect the distance to the wall while flying and playing sweeps.

In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

import rclpy
rclpy.init()

In [ ]:
def get_calib_function(calib="stepper"):
    from utils.calibration import get_calibration_function_median
    from utils.calibration import get_calibration_function_moving
    if calib == "flying":
        print("using flying")
        calib_function_median, freqs = get_calibration_function_moving(
            "2021_10_12_flying", motors="linear_buzzer_cont", fit_one_gain=False,
            #"2021_07_14_flying", motors="linear_buzzer_cont", fit_one_gain=False,
            appendix_list=["_new3", "_new4", "_new6"], 
        )
    elif calib == "stepper":
        print("using stepper dataset")
        calib_function_median, freqs = get_calibration_function_median(
            #"2021_07_08_stepper_fast",
            "2021_10_07_stepper_new_f",
            motors="all45000",
            mic_type="audio_deck",
            snr=5,
            fit_one_gain=False,
        )
    return calib_function_median

In [ ]:
def plot_positions(ax, positions_cm, wall_y_cm=100):
    from matplotlib import cm
    
    cmap = cm.get_cmap('inferno') 
    n_labels = 3
    label = None
    step = len(positions_cm) // n_labels
    for i, p in enumerate(positions_cm):
        if i % step == 0 or (i == len(positions_cm) - 1):
            label = f'position {i}'
        ax.scatter(*p[:2], color=cmap(i / len(positions_cm)), label=label)
        label=None
    ax.plot(positions_cm[:, 0], positions_cm[:, 1], color='k', ls=':')
    ax.axis('equal')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')
    ax.axhline(y=wall_y_cm, color='k', label='wall')
    ax.grid()
    ax.legend()

In [ ]:
def plot_matrix(yvalues, results_matrix, gt_values, xvalues=None, no_deco=False, angles=False, log=True):
    if xvalues is None:
        xvalues = gt_values
        
    estimates = get_estimates(results_matrix, yvalues)
    n_points = results_matrix.shape[1]
    fig, ax = plt.subplots()
    fig.set_size_inches(2 * FIGSIZE, FIGSIZE)
    im = pcolorfast_custom(
        ax,
        np.arange(n_points),
        yvalues,
        np.log10(results_matrix) if log else results_matrix,
        vmin=-3 if log else None,
    )
    ax.plot(
        np.arange(n_points) + 0.5,
        gt_values,
        color="white",
        label="ground truth",
        marker='x'
    )
    ax.set_ylim(min(yvalues), max(yvalues))
    ax.plot(
        np.arange(n_points) + 0.5,
        estimates,
        color="white",
        label="estimates",
        marker='o',
        ls='-'
    )
    if no_deco:
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        if log:
            add_colorbar(fig, ax, im, title="log-probability")
        else:
            add_colorbar(fig, ax, im, title="probability")
        if angles:
            ax.set_ylabel("estimated angle [deg]")
            ax.set_xlabel("real angle [cm]")
        else:
            ax.set_ylabel("estimated distance [cm]")
            ax.set_xlabel("real distance [cm]")
        #ax.set_title(f"appendix{appendix}, {n_points} measurements")
        leg = ax.legend(framealpha=0, loc='upper right')
        for text in leg.get_texts():
            plt.setp(text, color='w')
        ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
        ax.set_yticks(np.arange(min(yvalues), max(yvalues), step=10))
        ax.set_yticklabels(np.arange(min(yvalues), max(yvalues), step=10).astype(int))
    return fig, ax

def get_estimates(results_matrix, dists_cm):
    valid = np.any(results_matrix > 0, axis=0)
    d_estimates = dists_cm[np.argmax(results_matrix, axis=0)].astype(float)
    d_estimates[~valid] = np.nan
    return d_estimates

## 1. Single wall approach experiments

### Qualitative evaluation

In [ ]:
def extract_valid(row, wall_y_cm=100, forward_only=True):
    positions_cm = row.positions[:, :3] * 1e2 #get_positions_flying(row, start_distance_cm=95)
    yaws_deg = row.positions[:, 3]
    
    freqs = row.frequencies_matrix[0, :]
    
    values = np.abs(row.stft)[..., freqs>0] # n_times x n_mics x freqs
    freqs_valid = freqs[freqs>0]
    
    forward_indices = np.ones(positions_cm.shape[0], dtype=bool)
    if forward_only:
        forward_indices[:-1] &= (positions_cm[1:, 1] - positions_cm[:-1, 1] > 0)
    forward_indices &= positions_cm[:, 2] > 30
    means = np.sum(np.mean(values, axis=-1), axis=-1)
    crashs = np.where((means - np.mean(means)) > (2 * np.std(means)))[0]
    if len(crashs):
        print('crashs found:', crashs)
        forward_indices[crashs[0]:] = 0
    else:
        print('no crashs found')
        pass
    positions_cm = positions_cm[forward_indices, :]
    yaws_deg = yaws_deg[forward_indices]
    
    values = values[forward_indices, :]
    yaws_deg -= yaws_deg[0]
    positions_cm[:, :2] -= positions_cm[0, :2]
    
    distances = wall_y_cm - positions_cm[:, 1]
    return distances, freqs_valid, values, positions_cm, yaws_deg


def add_rect(ax, width, height, no_deco=False):
    rect = Rectangle((0, 0), width, height, facecolor='white', alpha=0.5)
    ax.add_patch(rect)
    if not no_deco:
        ax.text(width/2, 7, "used for\ncalibration", fontdict={'color':'white', 'ha':'center', 'va':'bottom'})

In [ ]:
from copy import deepcopy
import itertools
import time
from matplotlib.patches import Rectangle

from crazyflie_description_py.experiments import WALL_ANGLE_DEG
from utils.plotting_tools import pcolorfast_custom, FIGSIZE, save_fig
from utils.plotting_tools import add_colorbar
from utils.plotting_tools import plot_performance
from utils.inference import Inference, eps_normalize
from utils.estimators import DistanceEstimator
from utils.moving_estimators import MovingEstimator

# for live analysis
distances_cm = np.arange(100, step=5)
angles_deg = np.arange(360, step=30)

# for offline analysis
#distances_cm = np.arange(100, step=1)
#angles_deg = np.arange(360, step=2)


azimuth_deg = WALL_ANGLE_DEG
n_calib = 5
n_window = 5
bad_freqs = []

plot_dict = [
    {"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new3"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new4"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new6"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new7"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new8"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new10"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_new12"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_1"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_2"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_3"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_4"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_5"},
    #{"exp_name": "2021_10_12_flying", "motors": "linear_buzzer_cont", "appendix": "_6"},
]

wall_y_cm = 90

errors_df = pd.DataFrame(
    columns=["algorithm", "mics", "calib_method", "estimates", "distances", "appendix", "time"]
)

calib_method_list = ["on_the_fly"] 
#calib_method_list = ["stepper"]
#mic_list = [[1, 3], [0, 2], [0, 1, 3], [0, 1, 2, 3]]
mic_list = [[0, 1, 2, 3]]
algorithm_list = ["bayes"]#, "cost"]
no_deco = False

calib_flag = False
for params in plot_dict:
    exp_name, motors, appendix = params.values()

    results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
    row = results_df.loc[
        (results_df.appendix == appendix) & (results_df.motors == motors), :
    ].iloc[0]
    distances, freqs, values_unsorted, positions_cm, yaws_deg = extract_valid(row, wall_y_cm=wall_y_cm)
    n_points = values_unsorted.shape[0]
    
    if not calib_flag:
        calib_on_the_fly = np.median(values_unsorted[:n_calib, :, :], axis=0)
        calib_flag = True
    #print("calibration:", values_unsorted[:n_calib, :, :])
    std_values = np.std(values_unsorted[:n_calib, :, :], axis=0)

    plot_name_calib = f"plots/experiments/{exp_name}{appendix}_on_the_fly.pdf"
    fig_calib, axs_calib = plt.subplots(1, 5, sharey=True)
    fig_calib.set_size_inches(10, 3)
    axs_calib[0].set_ylabel(f"amplitude [-]")
    for m in range(4):
        axs_calib[m].plot(freqs, values_unsorted[:n_calib, m, :].T, color=f"C{m}", marker='o')
        axs_calib[m].set_title(f"mic{m}")
        axs_calib[4].errorbar(freqs, calib_on_the_fly[m], std_values[m], label=f"mic{m}")
        axs_calib[m].set_xlabel(f"frequency [Hz]")
    axs_calib[4].set_xlabel(f"frequency [Hz]")
    axs_calib[4].set_title(f"all mics")
    axs_calib[0].set_ylim(2, 12)
    [axs_calib[0].axvline(f, color='black') for fs in bad_freqs for f in fs]
    save_fig(fig_calib, plot_name_calib)

    fig, ax = plt.subplots()
    fig.set_size_inches(FIGSIZE, FIGSIZE)
    plot_positions(ax, positions_cm, wall_y_cm=wall_y_cm)
    plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"
    save_fig(fig, plot_name)

    inf_machine = Inference()
    dists_cm = DistanceEstimator.DISTANCES_M * 1e2
    azimuths_deg = DistanceEstimator.AZIMUTHS_DEG.astype(float)
    inf_machine.add_geometry([dists_cm[0], dists_cm[-1]], WALL_ANGLE_DEG)

    for calib_method in calib_method_list:
        if calib_method == "on_the_fly":
            calib_values = calib_on_the_fly
        else:
            calib_function = get_calib_function(calib_method)
            calib_values = calib_function(freqs)
        values_unsorted_calib = values_unsorted / calib_values

        for mics, algorithm in itertools.product(mic_list, algorithm_list):
            mics_str = str(mics).replace(" ", "")
            plot_name = f"plots/experiments/{exp_name}{appendix}_{algorithm}_{calib_method}_{mics_str}"

            results_matrix = np.full((len(dists_cm), n_points), np.nan)
            results_matrix_angles = np.full((len(azimuths_deg), n_points), np.nan)
            
            moving_estimator = MovingEstimator(n_window=n_window, distances_cm=distances_cm, angles_deg=angles_deg)
            results_matrix_angles_moving = np.full((len(moving_estimator.angles_deg), n_points), np.nan)
            results_matrix_moving = np.full((len(moving_estimator.distances_cm), n_points), np.nan)
            
            time_moving = 0
            time_single = 0
            for i in range(n_points):
                
                # important to reinitialize!
                distance_estimator = DistanceEstimator()

                inf_machine.add_data(
                    values_unsorted_calib[i],
                    freqs,
                )
                
                inf_machine.filter_out_freqs(freq_ranges=bad_freqs)
                
                t1 = time.time()
                diff_moving = {}
                for mic_idx in mics:
                    dist, prob_mic, diff = inf_machine.do_inference(algorithm, mic_idx)
                    distance_estimator.add_distribution(diff * 1e-2, prob_mic, mic_idx)
                    diff_moving[mic_idx] = (diff, prob_mic)

                __, prob = distance_estimator.get_distance_distribution(
                    distances_m=dists_cm * 1e-2, verbose=False, azimuth_deg=azimuth_deg,
                )
                time_single += int(1000*(time.time() - t1))
                
                t1 = time.time()
                moving_estimator.add_distributions(
                    diff_moving, position_cm=positions_cm[i], rot_deg=yaws_deg[i]
                )
                prob_moving, prob_angle_moving = moving_estimator.get_distributions(local=True)
                time_moving += int(1000*(time.time() - t1))
                
                dist_est_cm = dists_cm[np.argmax(prob_moving)]
                __, prob_angle = distance_estimator.get_angle_distribution(
                    distance_estimate_m=dist_est_cm*1e-2
                )
                
                results_matrix[:, i] = prob
                results_matrix_angles[:, i] = prob_angle
                results_matrix_moving[:, i] = prob_moving 
                results_matrix_angles_moving[:, i] = prob_angle_moving 
                
                continue
                # DEBUGGING ONLY
                dist = moving_estimator.get_joint_distribution()
                fig, ax = plt.subplots()
                pcolorfast_custom(ax, distances_cm, angles_deg, dist)
                ax.axvline(distances[i], color='white', ls=':')
                ax.axhline(90, color='white', ls=':')

            d_estimates = get_estimates(results_matrix, dists_cm)
            errors_df.loc[len(errors_df), :] = {
                "algorithm": algorithm,
                "mics": mics_str,
                "calib_method": calib_method,
                "estimates": d_estimates,
                "distances": distances,
                "appendix": appendix,
                "time": time_single / n_points 
            }
            fig, ax = plot_matrix(dists_cm, results_matrix, distances, no_deco=no_deco)
            if calib_method == 'on_the_fly':
                add_rect(ax, n_calib, max(dists_cm), no_deco=no_deco)
            #save_fig(fig, plot_name + "_probabilities.pdf")

            d_estimates_moving = get_estimates(results_matrix_moving, distances_cm)
            errors_df.loc[len(errors_df), :] = {
                "algorithm": algorithm + f"_win{n_window}",
                "mics": mics_str,
                "calib_method": calib_method,
                "estimates": d_estimates_moving,
                "distances": distances,
                "appendix": appendix,
                "time": time_moving / n_points 
            }
            fig, ax = plot_matrix(distances_cm, results_matrix_moving, distances, no_deco=no_deco)
            if calib_method == 'on_the_fly':
                add_rect(ax, n_calib, max(dists_cm), no_deco=no_deco)
            #save_fig(fig, plot_name + f"_win{n_window}_probabilities.pdf")
            
            
            gt_angles = np.full(len(distances), 90)
            
            angle_estimates = get_estimates(results_matrix_angles, azimuths_deg)
            fig, ax = plot_matrix(azimuths_deg, results_matrix_angles, gt_angles, no_deco=no_deco, angles=True, log=False)
            if calib_method == 'on_the_fly':
                add_rect(ax, n_calib, max(azimuths_deg), no_deco=no_deco)
            
            angle_estimates_moving = get_estimates(results_matrix_angles_moving, angles_deg)
            fig, ax = plot_matrix(angles_deg, results_matrix_angles_moving, 
                                  gt_angles, no_deco=no_deco, log=False, angles=True)
            if calib_method == 'on_the_fly':
                add_rect(ax, n_calib, max(angles_deg), no_deco=no_deco)

### Factor graph test

In [ ]:
from factor_graph.plot import plot_projections
import gtsam
from audio_gtsam.wall_backend import WallBackend

X = gtsam.symbol_shorthand.X
P = gtsam.symbol_shorthand.P

wall_backend = WallBackend(use_isam=True)
yaw_start = np.pi 

use_groundtruth = False
#use_groundtruth = True 

dist_matrix = results_matrix_moving
angle_matrix = results_matrix_angles_moving
n_estimates = 1
print(angle_matrix.shape, dist_matrix.shape)

# below doesn't work well
#dist_matrix = results_matrix
#angle_matrix = results_matrix_angles
#n_estimates = 2

d_estimates = []
d_gt = []

for i, prob_dists in enumerate(dist_matrix.T):
    t1 = time.time()
    if i < n_calib:
        continue
    
    position_cm = positions_cm[i, :]
    yaw = yaws_deg[i] / 180 * np.pi
    #print(f"yaw: {yaws_deg[i]:.1f}deg")
    #print(f"position: {position_cm[0], position_cm[1]}")
    pose_factor = wall_backend.add_pose(r_world=position_cm * 1e-2,  yaw=yaw)
    
    distance_gt = int(wall_y_cm - position_cm[1])
    if use_groundtruth:
        prob_dists = np.zeros(len(distances_cm))
        prob_dists[distance_gt] = 1.0
        
        prob_angles = np.zeros(len(angles_deg))
        prob_angles[angles_deg==90] = 1.0
    else:
        prob_angles = angle_matrix[:, i]
        
    wall_backend.add_planes_from_distributions(distances_cm, prob_dists, 
                                               angles_deg, prob_angles, 
                                               limit_distance=50,
                                               n_estimates=n_estimates, 
                                               verbose=False)
    
    wall_backend.check_wall(verbose=True)
    distance_estimate = wall_backend.get_distance_estimate()
    d_estimates.append(distance_estimate * 1e2 if distance_estimate else np.nan)
    d_gt.append(distance_gt)
    #print(f"time with isam={wall_backend.use_isam}: {(time.time() - t1)*1e3:.0f}ms")

errors_df.loc[len(errors_df), :] = {
    "algorithm": algorithm + f"_win{n_window}_factorgraph",
    "mics": mics_str,
    "calib_method": calib_method,
    "estimates": d_estimates,
    "distances": d_gt,
    "appendix": appendix,
    "time": time_moving / n_points 
}
   
#plot_projections(wall_backend.all_initial_estimates, axis_length=0.2, ls=":", perspective=False, side=True)
#plt.show()
wall_backend.get_results()
plot_projections(wall_backend.result, axis_length=0.1, perspective=False, side=True)
plt.show()

### quantitative evaluation

In [ ]:
def plot_random(axs, dists):
    np.random.seed(1)
    errors_rand = []
    for d in dists:
        d_rand = np.random.choice(range(7, 100))
        errors_rand.append(d_rand - d)
    axs[0].plot(dists, errors_rand, color='k', label='random', ls='', marker='x', markersize=2)
    axs[1].plot(sorted(np.abs(errors_rand)), np.linspace(0, 1, len(errors_rand)), color='k', ls='', label='random', marker='x', markersize=2)
    axs[1].legend(loc='lower right')

In [ ]:
from utils.plotting_tools import plot_performance, save_fig

#exp_name = '2021_10_12_flying'

#fname = f'../datasets/{exp_name}/results.pkl'
#errors_df = pd.read_pickle(fname)

#chosen_mics = '[0,1,2,3]'
#filtered_df = errors_df.loc[errors_df.mics==chosen_mics]

filtered_df = errors_df 
groupby = ['mics', 'algorithm']

total_err_dict = {}
for j, (labels, df) in enumerate(filtered_df.groupby(groupby, sort=False)):
    params = dict(zip(groupby, labels))
        
    err_dict = {}
    dist_dict = {}
    errors_all = []
    distances_all = []
    for i, row in df.iterrows():
        errors = np.array(row.estimates) - np.array(row.distances)
        key = ' '.join(row.drop(['distances', 'estimates', 'time'] + groupby).values)
        err_dict[key] = errors
        dist_dict[key] = row.distances
        errors_all += list(errors[~np.isnan(errors)])
        distances_all += list(np.array(row.distances)[~np.isnan(errors)])
        
    sort_idx = np.argsort(distances_all)
    total_err_dict[j] = {
        'errors': np.array(errors_all)[sort_idx],
        'distances': np.array(distances_all)[sort_idx],
        **params
    }
    
    title = ''
    for k,l in params.items():
        title = f"{title} {k}: {l}"

    fig, axs = plot_performance(err_dict, xs_dict=dist_dict, xlabel="distance [cm]", ylabel="error [cm]")
    plot_random(axs, range(10, 80))
    #fig, axs = plot_performance(err_dict, xlabel="distance [cm]", ylabel="error [cm]")
    fig.suptitle(title)
    axs[0].get_legend().set_visible(False)
    axs[0].set_ylim(-60, 60)
    axs[1].set_xlim(-2, 60)
    #save_fig(fig, f'plots/experiments/{exp_name}_{chosen_mics}_{params["algorithm"]}_cdf.pdf')

In [ ]:
max_dist = 40
err_dict = {}
dist_dict = {}
for total_dict in total_err_dict.values():
    key = total_dict['algorithm']
    errors = total_dict['errors']
    dists = total_dict['distances']
    err_dict[key] = errors[dists < 40]
    dist_dict[key] = dists[dists < 40]
    
fig, axs = plot_performance(err_dict, xs_dict=dist_dict, xlabel="distance [cm]", ylabel="error [cm]", marker_flag=False)
axs[0].get_legend().set_visible(False)
plot_random(axs, range(10, max_dist))
fig.suptitle(f'overall performance up to {max_dist}cm')
save_fig(fig, f'plots/experiments/{exp_name}_cdf.pdf')

## 2. Multi-wall approach experiments 

In [ ]:
exp_name = "2021_11_23_demo"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

# glass wall
#appendix = "hover1" # works super well! 
#appendix = "hover3" # looks good! 
#appendix = "hover5" # looks quite good! 
#appendix = "hover6" # looks ok (from here on, velocity is increased)
appendix = "hover8" # works well (glass wall), but only detects one wall

# normal wall
#appendix = "hover9" # works not great
#appendix = "hover10" # works not great
#appendix = "hover11" # works okay
#appendix = "hover12" # works okay 
wall_y_cm = 100

In [ ]:
exp_name = "2022_01_25"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

appendix = "test24"  # works ok
#appendix = "test25"  # doesn't work

wall_y_cm = 80

In [ ]:
from crazyflie_demo.wall_detection import FLYING_HEIGHT_CM, N_CALIBRATION, N_WINDOW
from crazyflie_demo.wall_detection import WallDetection, LOCAL_DIST_RANGE_CM

wall_detection = WallDetection()

calib_method = "on_the_fly"
mics = [0, 1, 2, 3]
#mics = [3] # towards
#mics = [0] # towards
#mics = [2] # back
algorithm = "bayes" #"cost" #"bayes"

azimuth_deg = WALL_ANGLE_DEG

row = results_df.loc[results_df.appendix == appendix,:].iloc[0]

#distances, freqs, magnitudes, positions_cm, yaws_deg = extract_valid(row, 
#                                                                     wall_y_cm=wall_y_cm, 
#                                                                     forward_only=False)
positions_cm = row.positions[:, :3] * 1e2
flying = (positions_cm[:, 2] > FLYING_HEIGHT_CM) & (positions_cm[:, 2] < 100)
print("not flying:", np.where(~flying)[0])
positions_cm = positions_cm[flying, :]
#print(positions_cm)
yaws_deg = row.positions[flying, 3]
freqs = row.frequencies_matrix[0, :]
magnitudes = np.abs(row.stft[flying][:, :, freqs>0])
signals_f = row.stft[flying][:, :, freqs>0]
freqs = freqs[freqs > 0]
times = row.seconds[flying]

# because wall is at 90 degrees.
# TODO: make this statement more general
distances = wall_y_cm - positions_cm[:, 1]

n_timesteps = magnitudes.shape[0] #-1

wall_backend = WallBackend(use_isam=True)
d_estimates = []
colors = []
angle_estimates = []

calib_values = np.median(magnitudes[:N_CALIBRATION, :, :], axis=0)
std_values = np.std(magnitudes[:N_CALIBRATION, :, :], axis=0)

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
plot_positions(ax, positions_cm, wall_y_cm=wall_y_cm)
plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"
#save_fig(fig, plot_name)

inf_machine = Inference()
inf_machine.add_geometry(LOCAL_DIST_RANGE_CM, WALL_ANGLE_DEG)

magnitudes_calib = magnitudes / calib_values[None, :, :]

results_matrix_angles = np.full((len(angles_deg), n_timesteps), np.nan)
results_matrix_moving = np.full((len(distances_cm), n_timesteps), np.nan)

moving_estimator = MovingEstimator(n_window=n_window) #distances_cm=distances_cm, angles_deg=angles_deg)

calib = True
for i in range(n_timesteps):
    
    position_cm = positions_cm[i, :] 

    if i == N_CALIBRATION + 1:
        print("change state")
        calib = False
        
    res = wall_detection.listener_callback_offline(
        signals_f[i].T, freqs, positions_cm[i], yaws_deg[i], calib=calib, timestamp=times[i]*1e3
    )
    
    
    if i <= N_CALIBRATION:
        d_estimates.append(None)
        angle_estimates.append(None)
        colors.append("k")
        continue
        
    if res is None:
        print("no result yet!")
        
    __, __, prob_moving_dist, prob_moving_angle = res

        
    results_matrix_moving[:, i] = prob_moving_dist
    results_matrix_angles[:, i] = prob_moving_angle
    
    yaw = yaws_deg[i] / 180 * np.pi
    pose_factor = wall_backend.add_pose(r_world=positions_cm[i] * 1e-2,  yaw=yaw, verbose=False)
    #wall_backend.add_planes_from_distributions(distances_cm, prob_moving_dist, angles_deg, prob_moving_angle, n_estimates=1, verbose=True)
    
    wall_backend.add_plane_from_distances(distances_cm, prob_moving_dist, verbose=False)
    #wall_backend.check_wall(verbose=True)
    
    d_estimate = wall_backend.get_distance_estimate()
    angle_deg = wall_backend.get_angle_estimate()
    d_estimates.append(d_estimate * 1e2 if d_estimate else None)
    colors.append(f"C{wall_backend.plane_index+1}")
    angle_estimates.append(angle_deg)

gt_angles = np.full(results_matrix_angles.shape[1], 90)
fig, ax = plot_matrix(angles_deg, results_matrix_angles, gt_angles, xvalues=times[:n_timesteps], 
                      no_deco=no_deco, angles=True, log=False)
ax.scatter(np.arange(len(angle_estimates)), angle_estimates, color=colors)
add_rect(ax, N_CALIBRATION, max(angles_deg), no_deco=no_deco)

fig, ax = plot_matrix(distances_cm, results_matrix_moving, distances[:n_timesteps], xvalues=times[:n_timesteps], 
                      no_deco=no_deco)
ax.scatter(np.arange(len(d_estimates)), d_estimates, color=colors)
add_rect(ax, N_CALIBRATION, max(distances_cm), no_deco=no_deco)
#save_fig(fig, plot_name + f"_win{n_window}_probabilities.pdf")

In [ ]:
angle_plot = np.array(angle_estimates)
distance_plot = np.array(d_estimates)

fig, ax = plt.subplots()
ax.scatter(range(len(angle_plot)), angle_plot)
ax.scatter(np.where(angle_plot == None)[0], np.ones(np.sum(angle_plot == None)))
fig, ax = plt.subplots()
ax.scatter(range(len(distance_plot)), distance_plot)
ax.scatter(np.where(distance_plot == None)[0], np.ones(np.sum(distance_plot == None)))

In [ ]:
plot_projections(wall_backend.result, perspective=False, top=True, side=False, axis_length=0.1)

In [ ]:
### alternative angle calculation: geometric averaging based on mic level differences

In [ ]:
mic_angles = np.array([90, 180, -90, 0])
mics = [0, 1, 2, 3]

total = np.sum(magnitudes[:, :, :]**2, axis=(0, 2))
powers = np.sum(magnitudes[:, mics]**2, axis=2) / total[None, :]

fig = plt.figure()
for i, mic in enumerate(mics): #range(powers.shape[1]):
    plt.plot(range(powers.shape[0]), powers[:, i], label=f"mic{mic}")
plt.legend()
fig.set_size_inches(10, 5)

fig = plt.figure()
norm = powers[:] / np.mean(powers, axis=1)[:, None]
print(norm[0, :])

angle_vectors = np.array([[np.cos(a / 180 * np.pi), np.sin(a / 180 * np.pi)] for a in mic_angles[mics]])
vectors = norm.dot(angle_vectors)
angles = np.arctan2(vectors[:, 1], vectors[:, 0]) * 180 / np.pi
weights = np.linalg.norm(vectors, axis=1)

significant = np.where(weights > 0.2)[0]

fig = plt.figure()
plt.plot(range(powers.shape[0]), angles, label=f"all")
plt.scatter(significant, angles[significant], label=f"significant")
fig.set_size_inches(10, 5)

fig = plt.figure()
plt.plot(range(powers.shape[0]), weights, label=f"mic{mic}")
fig.set_size_inches(10, 5)

# WIP: flying experiments with timestamps for each amplitude (snr=3)

In [ ]:
MIN_Z_CM = 30 # minimum flying height

def plot_plositions(row, min_time=None, max_time=None, max_dist=None):
    import seaborn as sns
    positions_cm = row.positions[:, :3] * 100
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = np.ones_like(row.seconds, dtype=np.bool)
    if min_time is not None:
        mask_time = (row.seconds > min_time) 
    else:
        mask_time = positions_cm[:, 2] > MIN_Z_CM
    if max_time is not None:
        mask_time = mask_time & (row.seconds < max_time)
    else:
        mask_time = positions_cm[:, 2] > MIN_Z_CM
        
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    if max_dist is not None:
        axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')
    plt.tight_layout()
    return fig, axs

def plot_audio(row, mic_idx=0):
    from utils.plotting_tools import pcolorfast_custom
    all_frequencies = row.freqs
    spec = row.spectrogram[:, mic_idx, :]
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    
    # mark too long measurements gray
    max_diff = 1
    diff = row.seconds[1:] - row.seconds[:-1]
    indices = np.where(diff>max_diff)[0]
    endings = row.seconds[:-1][indices]
    diff_average = np.mean(diff[diff<max_diff])
    seconds = row.seconds
    for counter, i in enumerate(indices):
        new_time = seconds[i+counter]+diff_average
        seconds = np.insert(seconds, i+counter+1, new_time)
        spec = np.insert(spec, i+counter+1, np.nan, axis=1)
    
    pcolorfast_custom(ax, seconds, all_frequencies, np.abs(spec))
    
    xticks = np.arange(0, row.seconds.max(), step=5)
    ax.set_xticks(xticks); ax.set_xticklabels(xticks)
    yticks = np.arange((np.round(row.freqs.min()//1000)+1)*1000, 
                        row.freqs.max(), step=1000)
    ax.set_yticks(yticks); ax.set_yticklabels(yticks)
    ax.set_ylabel('frequency [Hz]')
    ax.set_xlabel('seconds [s]')
    return fig, ax

linestyles = {"estimated": "-", "theo": ":", "theo_corr": "-.", "measured": "-"}
colors = {"estimated": "C0", "theo": "black", "theo_corr": "black", "measured":"C0"}
keys = {"estimated": "estimated", "theo": "theo, vertical", "theo_corr": "theoretical", "measured":"measured"}


def change_order(axs_all, mean_distances, xlabel="distance", unit="cm", ylabel="probability [-]", title=True):
    sorted_idx = np.argsort(mean_distances)
    positions = [ax.get_position() for ax in axs_all]
    for i, idx in enumerate(sorted_idx):
        axs_all[idx].set_position(positions[i])
        if title:
            axs_all[idx].set_title(f"{mean_distances[idx]:.0f}{unit}")
    [ax.get_yaxis().set_visible(False) for ax in axs_all[sorted_idx[1:]]]
    axs_all[sorted_idx[len(sorted_idx)//2]].set_xlabel(f"{xlabel} [{unit}]")
    axs_all[sorted_idx[0]].set_ylabel(ylabel)
    axs_all[sorted_idx[-1]].legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
    return axs_all

In [ ]:
exp_name = '2021_05_04_flying'; # snr = 3
fname = f'../datasets/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run csv_or_wav_parser.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

df_total.sort_values(by='appendix', inplace=True)

from utils.geometry import Context
context = Context.get_crazyflie_setup(dim=2)
fig, ax = plt.subplots()
context.plot(ax=ax)
fig.set_size_inches(3, 3)


min_time = None #4 #None
max_time = None #14 #None
max_dist = None

#starting_distance = 65.63 # 42+29.7−6.07 
starting_distance = 100

fig_total, ax_total = plt.subplots()
fig_total.set_size_inches(3, 3)
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time, max_time, max_dist)
    
    x = row.positions[:, 0] * 100
    y = row.positions[:, 1] * 100 - starting_distance
    z = row.positions[:, 2] * 100
    
    x = x[~np.isnan(x)] 
    y = y[~np.isnan(y)]
    
    x = x[z[~np.isnan(z)] > MIN_Z_CM]
    y = y[z[~np.isnan(z)] > MIN_Z_CM]
    
    #ax_total.scatter(x, y, s=10.0, label=row.appendix)
    ax_total.plot(x, y, marker='o', label=row.appendix.replace('_', ''))
    #mask2 = (y > -130) & (y < -100)
    #mask3 = (y > -100) & (y < -70)
    #mask4 = (y > -70)
    #for mask in [mask2, mask3, mask4]:
    #    ax_total.scatter(x[mask], y[mask], s=10.0)
    #save_fig(fig, f'plots/experiments/{exp_name}{row.appendix}_movement', extension='.png')
ax_total.axis('equal')
ax_total.set_xlabel('x [cm]')
ax_total.set_ylabel('y [cm]')
ax_total.axhline(0, color='k', label='wall')
ax_total.legend(bbox_to_anchor=[1.0, 1.0], loc='upper left')
#save_fig(fig_total, f'plots/experiments/{exp_name}_pos.png')

In [ ]:
## audio analysis

from utils.frequency_analysis import add_spectrogram

df_total = df_total.assign(spectrogram=None,freqs=None)
df_total = df_total.apply(add_spectrogram, axis=1)
print(df_total.columns)

mic_idx = 0

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    
    #complicated spectrogram
    fig, ax = plot_audio(row, mic_idx=mic_idx)
    #ax.set_ylim([2800, 5000])
    #save_fig(fig, f'plots/experiments/{exp_name}{row.appendix}_spec')
    
    #if not row.appendix in ["_bin5_thirdtry", "_bin6"]:
    #    continue
    #fig, ax = plt.subplots()
    #ax.pcolorfast(row.seconds, row.freqs, np.log10(np.abs(row.spectrogram[:-1, mic_idx, :-1])))
    #ax.set_title(row.appendix)

In [ ]:
## algorithm performance
from utils.calibration import get_calibration_function_median
from utils.inference import Inference
from utils.plotting_tools import plot_df

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
calib_function, calib_freq = get_calibration_function_median(
    "2021_04_30_stepper", "audio_deck", ax=ax, snr=3, motors=0#fit_one_gain=True 
)
inf_machine = Inference()
inf_machine.add_calibration_function(calib_function)


from utils.dataset_parameters import kwargs_datasets
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

kwargs = kwargs_datasets[exp_name]["audio_deck"]
azimuth_deg = WALL_ANGLE_DEG

distance_range = [7, 50]
freq_range = [3000, 5000]

fig_df, ax_df = plot_df(distance_range, azimuth_deg=azimuth_deg)

inf_machine.add_geometry(distance_range, azimuth_deg)

In [ ]:
from copy import deepcopy
from itertools import cycle

from utils.data_collector import DataCollector
from utils.estimators import DistanceEstimator, get_estimate
from utils.inference import eps_normalize
from utils.simulation import get_freq_slice_theory

eps = 1e-5  # for plotting only
max_plot_distance = 110
algorithm = "bayes"
# algorithm = "cost"
normalize = True  # normalize probas before combining
method = "sum"  # method used to combine

nominal_ds = [60, 40, 20]

freqs_calib = np.linspace(
    np.min(row.frequencies_matrix), np.max(row.frequencies_matrix), 50
)
f_calib_all = calib_function(freqs_calib)

chosen_mics = None

plot_idx = 0
mean_distances = []

fig_df, ax_df = plot_df([7, max_plot_distance], azimuth_deg=azimuth_deg)
fig_all, axs_all = plt.subplots(1, 9)
fig_res, axs_res = plt.subplots(1, 9, sharex=True)
fig_all.set_size_inches(15, 3)
fig_res.set_size_inches(15, 2)

#row = df_total.loc[df_total.appendix == "_22"].iloc[0]
for i_row, row in df_total.iterrows():
    
    nominal_distances = cycle(nominal_ds)
    data_collector = DataCollector(exp_name=exp_name)
    print('treating', row.appendix)
    print('==================')
    
    flying_time_indices = list(np.where(row.positions[:, 2] * 1e2 > MIN_Z_CM)[0])

    count = 0
    sweep_complete = False

    for i in (flying_time_indices + [-1]):
        if i == row.stft.shape[0]:
            sweep_complete = True
            print("reached end of dataset")
        elif i == -1:
            sweep_complete = True
            print("reached end of dataset")
        elif row.bin_selection < 5:
            signals_f = row.stft[i]
            frequencies = row.frequencies_matrix[i]
            sweep_complete = data_collector.next_fslice_ready(
                signals_f, frequencies, verbose=False
            )
        elif i > 0:
            sweep_complete = True
        else:
            print("not ready yet")

        if sweep_complete:
            nominal_distance = next(nominal_distances)
            (
                f_slice,
                freqs,
                stds,
                distances,
            ) = data_collector.get_current_frequency_slice(verbose=False, df_cleanup=False)

            print("treating new frequency slice after", count)
            count = 0


            rel_distances = starting_distance - distances - nominal_distance
            inf_machine.add_data(
                deepcopy(f_slice), freqs, stds, deepcopy(rel_distances)
            )
            inf_machine.calibrate()

            # raw data, for plotting only
            freqs = inf_machine.values[inf_machine.valid_idx]
            distances = distances[inf_machine.valid_idx]

            distance_corr = starting_distance - distances
            mean_distance = np.nanmean(distance_corr)
            rel_distances = distance_corr - nominal_distance  # relative movement

            if mean_distance < max_plot_distance:
                ax_df.scatter(distance_corr, freqs, color=f"C{plot_idx}")
                ax_df.axvline(
                    mean_distance, color="black", ls=":", label="mean distance"
                )

            distance_estimators = {
                "measured": DistanceEstimator(),
                "theo": DistanceEstimator(),
                "theo_corr": DistanceEstimator(),
            }

            # treat measured data
            for i_mic in range(f_slice.shape[0]):
                dists, proba, diff = inf_machine.do_inference(
                    mic_idx=i_mic, algorithm=algorithm, normalize=normalize
                )
                distance_estimators["measured"].add_distribution(
                    diff * 1e-2, proba, i_mic
                )

            # treat theorectical data
            f_theo = get_freq_slice_theory(
                freqs, distance_cm=mean_distance, azimuth_deg=azimuth_deg
            ).T  # n_mics x n_freqs
            inf_machine.add_data(f_theo, freqs)
            for i_mic in range(f_theo.shape[0]):
                dists_theo, proba_theo, diff_theo = inf_machine.do_inference(
                    mic_idx=i_mic,
                    algorithm=algorithm,
                    normalize=normalize,
                    calibrate=False,
                )
                distance_estimators["theo"].add_distribution(
                    diff_theo * 1e-2, proba_theo, i_mic
                )

            f_theo_corr = get_freq_slice_theory(
                freqs, distance_cm=distance_corr, azimuth_deg=azimuth_deg
            ).T
            inf_machine.add_data(f_theo_corr, freqs, distances=rel_distances)
            for i_mic in range(f_theo_corr.shape[0]):
                (
                    dists_theo_corr,
                    proba_theo_corr,
                    diff_theo_corr,
                ) = inf_machine.do_inference(
                    mic_idx=i_mic,
                    algorithm=algorithm,
                    normalize=normalize,
                    calibrate=False,
                )
                distance_estimators["theo_corr"].add_distribution(
                    diff_theo_corr * 1e-2,
                    proba_theo_corr - np.mean(proba_theo_corr),
                    i_mic,
                )

            for key, distance_estimator in distance_estimators.items():

                distance_total, proba_total = distance_estimator.get_distance_distribution(method=method, chosen_mics=chosen_mics, azimuth_deg=azimuth_deg)


                if mean_distance < max_plot_distance:
                    axs_all[plot_idx].plot(
                        distance_total * 1e2,
                        eps_normalize(proba_total, eps),
                        label=keys[key],
                        ls=linestyles[key],
                        color=f"C{plot_idx}",
                    )

                    d = get_estimate(distance_total * 1e2, proba_total)
                    axs_res[plot_idx].axvline(
                        d, label=keys[key], ls=linestyles[key], color=f"C{plot_idx}",
                    )

            if mean_distance < max_plot_distance:
                axs_all[plot_idx].axvline(
                    mean_distance, color="k", ls=":", label="mean distance"
                )
                axs_all[plot_idx].set_yscale("log")

                axs_res[plot_idx].axvline(
                    mean_distance, color="k", ls=":", label="mean distance"
                )
                axs_res[plot_idx].scatter(distance_corr, freqs, color=f"C{plot_idx}")
                axs_res[plot_idx].set_ylim(3000, 5000)

                mean_distances.append(mean_distance)
                plot_idx += 1
            else:
                print('not plotting', mean_distance)

        if i < row.stft.shape[0]:
            signals_f = row.stft[i]
            frequencies = row.frequencies_matrix[i]
        
            mode = "maximum" if row.bin_selection < 5 else "all"
            data_collector.fill_from_signal(
                signals_f,
                frequencies,
                distance_cm=row.positions[i, 1] * 1e2,
                time=row.seconds[i],
                mode=mode,
            )
            count += 1
        else:
            print('done')


# sort subplots according to distance
axs_all = change_order(axs_all, mean_distances)
axs_res = change_order(axs_res, mean_distances, ylabel="frequency [Hz]", title=False)

#save_fig(fig_df, f"plots/experiments/{exp_name}_df.png")  # , extension="png")
#save_fig( fig_all, f"plots/experiments/{exp_name}_mics{chosen_mics}_all.png" )  # , extension="png")
#save_fig( fig_res, f"plots/experiments/{exp_name}_mics{chosen_mics}_res.png" )  # , extension="png")